In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
%matplotlib inline

def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [2]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

# Try scratched Vgg

In [5]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [6]:
SHAPE = (64, 64)
CHANNEL = 3
    
data = generate_data_set_for_training(shape=SHAPE, channel=CHANNEL)
# Extract Train, val from data
X_train = data['X_train']
X_val = data['X_val']
# Format as the input of VGG
y_train = list(data['y_train'].T)
y_val = list(data['y_val'].T)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [10]:
_,row, col,channel = X_train.shape
digLen = 5 # including category 0
numDigits = 11
epochs = 50
batch_size = 64

vgg16Model = VGG16(include_top = False,
                   weights = None)
vgg16Model.summary()
ptInput = keras.Input(shape = (row,col,channel), name  = 'vgg16Scratch')
vgg16 = vgg16Model(ptInput)

# vgg16 = Conv2D(64,(3, 3), activation ='relu', padding='same')(input)
# vgg16 = Conv2D(64,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(128,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(128,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(256,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(256,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)

vgg16 = Flatten()(vgg16)
vgg16 = Dense(512, activation='relu')(vgg16)
vgg16 = Dense(512, activation='relu')(vgg16)
# vgg16 = Dense(1000, activation='relu')(vgg16)
vgg16 = Dropout(0.5)(vgg16)

numd_SM = Dense(digLen,    activation='softmax',name = 'num')(vgg16)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(vgg16)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(vgg16)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(vgg16)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(vgg16)
numB_SM = Dense(2,         activation='softmax',name = 'nC')(vgg16)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM, numB_SM]

vgg16 = keras.Model(inputs = ptInput, outputs = out)

callback = []
optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/vgg16.classifier.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 3,
                                              cooldown= 0,
                                              min_lr = 0.000001)
# tb = keras.callbacks.TensorBoard(log_dir='logs', write_graph=True, write_images=True)
es = keras.callbacks.EarlyStopping(monitor= 'val_loss',
                                   min_delta=0.00000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)
vgg16.summary()

vgg16.compile(loss = 'sparse_categorical_crossentropy',
              optimizer= optim,
              metrics=  ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [11]:
vgg16History = vgg16.fit(x = X_train,
                         y = y_train,
                         batch_size = batch_size,
                         epochs=epochs,
                         verbose=1,
                         shuffle = True,
                         validation_data = (X_val, y_val),
                         callbacks = callback)

print(vgg16History.history.keys())
modName = 'vgg16_Scratch'
print(vgg16History.history.keys())

Train on 69076 samples, validate on 17269 samples
Epoch 1/50
69076/69076 [==============================] - 83s 1ms/step - loss: 2.7769 - num_loss: 0.4649 - dig1_loss: 0.8123 - dig2_loss: 0.8720 - dig3_loss: 0.4702 - dig4_loss: 0.1144 - nC_loss: 0.0431 - num_acc: 0.8108 - dig1_acc: 0.7100 - dig2_acc: 0.7101 - dig3_acc: 0.8967 - dig4_acc: 0.9842 - nC_acc: 0.9832 - val_loss: 2.6065 - val_num_loss: 0.4268 - val_dig1_loss: 0.7792 - val_dig2_loss: 0.8566 - val_dig3_loss: 0.4549 - val_dig4_loss: 0.0888 - val_nC_loss: 2.1846e-04 - val_num_acc: 0.8244 - val_dig1_acc: 0.7215 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 1.0000

Epoch 00001: loss improved from inf to 2.77693, saving model to saved_models/vgg16.classifier.hdf5
Epoch 2/50
69076/69076 [==============================] - 83s 1ms/step - loss: 2.6230 - num_loss: 0.4314 - dig1_loss: 0.7842 - dig2_loss: 0.8561 - dig3_loss: 0.4538 - dig4_loss: 0.0954 - nC_loss: 0.0022 - num_acc: 0.8229 - dig1_acc: 0.71

In [12]:





%load_ext autoreload
%autoreload 2
from utils.model_evaluation_utils import evaluate_model_performance

evaluate_model_performance(fitted_model=vgg16History, 
                           model_name = modName,
                           data=data,
                           model=vgg16
                          )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train loss: 25.18283789730409
number_of_digits digit1 digit2 digit3 digit4
Train accuracy: [61.36284671955528, 61.46128901499798, 71.20128554056402, 89.79674561352712, 98.57548207771151, 61.36284671955528]
Train sequence accuracy: 61.42944003706064
Validation loss: 25.31444076108659
number_of_digits digit1 digit2 digit3 digit4
Validation accuracy: [61.18478197926921, 61.27164282818924, 71.0637558631073, 89.58827957611905, 98.65076148010886, 61.18478197926921]
Validation sequence accuracy: 61.24268921188256
Test loss: 62.18777540327487
number_of_digits digit1 digit2 digit3 digit4
Test accuracy: [0.0, 0.13776213072095514, 29.710699525485996, 85.2211847543242, 99.1045461503138, 0.0]
Test sequence accuracy: 0.09949487218735649


{'train_acc': [61.36284671955528,
  61.46128901499798,
  71.20128554056402,
  89.79674561352712,
  98.57548207771151,
  61.36284671955528],
 'test_acc': [0.0,
  0.13776213072095514,
  29.710699525485996,
  85.2211847543242,
  99.1045461503138,
  0.0],
 'val_acc': [61.18478197926921,
  61.27164282818924,
  71.0637558631073,
  89.58827957611905,
  98.65076148010886,
  61.18478197926921],
 'train_seq_acc': 61.42944003706064,
 'test_seq_acc': 0.09949487218735649,
 'val_seq_acc': 61.24268921188256,
 'train_score': [25.18283789730409,
  6.227573267731105,
  6.211707909262252,
  4.641805765797112,
  1.6445714459517689,
  0.2296062318839852,
  6.2275732623190905,
  0.6136284671990042,
  0.6146128901534312,
  0.7120128554021886,
  0.8979674561352713,
  0.9857548207771151,
  0.6136284671990042],
 'test_score': [62.18777540327487,
  16.11809539794922,
  16.095892663135583,
  11.329298380558768,
  2.3820647825494743,
  0.14433117896689987,
  16.11809539794922,
  0.0,
  0.0013776213072095515,
  0.2